<span style='font-size:20.5px; letter_spacing: 1px'>The project is an AB-test. Project's goal: to determine any causal effect in users behaviour after being exposed to a <b>old_page</b> design or <b>new_page</b> design, which is captured by <b>converted</b> column</span>

In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

<h2>1. Let's take a look at the data. Figure out size of both control and treatment groups, possibly clean the data, convert columns into needed data types</h2>

In [122]:
#read data
df = pd.read_csv('ab_data.csv')

In [ ]:
#overview
for column in df.columns:
    print(column)
    print(df[column].unique())

<span><b>Ensure user ids do not dublicate</b><span>

In [169]:

counter = df['user_id'].value_counts()

#counter
valid_users = pd.DataFrame(counter[counter == 1].index, columns=['user_id'])
df =df.merge(valid_users, on='user_id')

In [170]:
counter

851104    1
701718    1
646178    1
715222    1
886963    1
         ..
683486    1
671693    1
875919    1
701344    1
715931    1
Name: user_id, Length: 286690, dtype: int64

In [171]:
#determine size of the groups
df['group'].value_counts()

treatment    143397
control      143293
Name: group, dtype: int64

In [124]:
#ration of the groups
len(df[df['group'] == 'treatment']) / len(df[df['group'] == 'control']) - 1

0.0005027105609978211

In [125]:
#how long lasted the experiment 
print(df[df['group'] == 'treatment']['timestamp'].min())
print(df[df['group'] == 'treatment']['timestamp'].max())

2017-01-02 13:42:05.378582
2017-01-24 13:41:44.097174


In [126]:
#overview
df['timestamp'].describe()

count                         294478
unique                        294478
top       2017-01-21 22:11:48.556739
freq                               1
Name: timestamp, dtype: object

In [127]:
#convert timestamps into a proper datatype
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [128]:
#df['timestamp'].astype('str')

<h3>start of the test...
</h3>

In [129]:
df_control_converted = df[(df['group'] == 'control') & (df['converted'] == 1)]

In [130]:
df_control_unconverted = df[(df['group'] == 'control') & (df['converted'] == 0)]

In [131]:
df_treatment_converted = df[(df['group'] == 'treatment') & (df['converted'] == 1)]
df_treatment_unconverted = df[(df['group'] == 'treatment') & (df['converted'] == 0)]

In [132]:
control_ratio = len(df_control_converted)/len(df_control_unconverted)

In [133]:
treatment_ratio = len(df_treatment_converted) / len(df_treatment_unconverted)

In [172]:
#treatment and control groups are almost symmetrical, which allows for correct analysis
treatment_ratio/control_ratio

0.9860522169014425

In [136]:
#creating a week column
import datetime
df['week'] = df['timestamp'].dt.isocalendar().week

In [137]:
treatment_data = df.loc[df['group'] != 'control']

In [138]:
df.groupby('group')['converted'].value_counts()

group      converted
control    0            126073
           1             17220
treatment  0            126372
           1             17025
Name: converted, dtype: int64

Transform the data into 2x2 matrix for chi test

In [139]:
observed_values = pd.crosstab(df['group'], df['converted']).values
observed_values

array([[126073,  17220],
       [126372,  17025]], dtype=int64)

In [140]:
control_rate = 17723 / (17723 + 129479)
treatment_rate = 17512 / (129762 + 17514)

In [141]:
control_rate


0.12039917935897611

In [142]:
treatment_rate

0.11890599961976153

In [143]:
from scipy.stats import chi2_contingency, chi2
a = 0.05

In [144]:
chi2_statistics, p_value, dof, expected_values = chi2_contingency(observed_values, correction=False)

In [145]:
print(chi2_statistics, p_value)

1.426794609399621 0.23228827305833816


In [146]:
h_0 = 'no relationship'
h_1 = 'relationship exists'
if chi2_statistics > a:
    print(h_1, ':', 'chi2', chi2_statistics, 'p_value', p_value)
elif chi2_statistics < a:
    print(h_0, ':', 'chi2', chi2_statistics, 'p_value', p_value)

relationship exists : chi2 1.426794609399621 p_value 0.23228827305833816


In [147]:
critical_value = chi2.ppf(1 - a, dof)
critical_value

3.841458820694124

<H1>2. Lets us look at the conversion rate broken down weekly</H1>

In [148]:
start_time = pd.to_datetime(df['timestamp'].min())
end_time = pd.to_datetime(df['timestamp'].max())
data_duration_seconds = (end_time - start_time).total_seconds()
data_duration_days = data_duration_seconds / (24 * 3600)
print(f'data collected for {data_duration_days} days' )

data collected for 21.999873633414353 days


In [149]:
df

,user_id,timestamp,group,landing_page,converted,week
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,3
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,2
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,2
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,1
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,3
...,...,...,...,...,...,...
286685,751197,2017-01-03 22:28:38.630509,control,old_page,0,1
286686,945152,2017-01-12 00:51:57.078372,control,old_page,0,2
286687,734608,2017-01-22 11:45:03.439544,control,old_page,0,3
286688,697314,2017-01-15 01:20:28.957438,control,old_page,0,2


In [150]:
df['week'].value_counts()

2    91380
3    91056
1    83745
4    20509
Name: week, dtype: Int64

In [214]:
#calculating the ration between control and treatment groups week by week
n_f_w = 4
experiment_data = df[df['week'] <= n_f_w]

In [215]:
control = experiment_data[experiment_data['group'] == 'control']
treatment = experiment_data[experiment_data['group'] == 'treatment']

control_converted = round(control['converted'].sum() * 100/control['converted'].count(), 3)
treatment_converted = round(treatment['converted'].sum() * 100/treatment['converted'].count(), 3)

lift = round(control_converted - treatment_converted, 3)

In [216]:
#the actual difference ratio
lift

0.144

In [217]:
#buiding a chi test for weekly effect
observed_values2 = pd.crosstab(experiment_data['group'], experiment_data['converted']).values
observed_values2

array([[126073,  17220],
       [126372,  17025]], dtype=int64)

In [218]:
chi2_statistics, p_value, dof, expected_values = chi2_contingency(observed_values2, correction=False)
print(chi2_statistics, p_value)

1.426794609399621 0.23228827305833816


In [219]:
h_0 = 'no relationship'
h_1 = 'relationship exists'
if chi2_statistics > a:
    print(h_1, ':', 'chi2', chi2_statistics, 'p_value', p_value)
elif chi2_statistics < a:
    print(h_0, ':', 'chi2', chi2_statistics, 'p_value', p_value)

relationship exists : chi2 1.426794609399621 p_value 0.23228827305833816


<span style='font-size: 25px'>Second week exposure...</span>

The conversion ration fluctuates by 0,01%-0,015%, with p_value decreasing with every week. That tells us, that the most conversions occurred during the first week, and the speed of conversion was slowing down throughout the whole experiment
